In [1]:
import requests
import json
import pandas as pd

# Example: GDP (current USD) for USA
url = "https://api.worldbank.org/v2/country/USA/indicator/NY.GDP.MKTP.CD?format=json&per_page=1000"
response = requests.get(url)

data = response.json()

# Data is inside index 1 → convert to DataFrame
df = pd.DataFrame(data[1])
df.head()



StatementMeta(, 493837e5-74ea-4355-89f2-530603fe6e7d, 3, Finished, Available, Finished)

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'US', 'value': 'United States'}",USA,2024,2.918489e+13,,,0
1,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'US', 'value': 'United States'}",USA,2023,2.772071e+13,,,0
2,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'US', 'value': 'United States'}",USA,2022,2.600689e+13,,,0
3,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'US', 'value': 'United States'}",USA,2021,2.368117e+13,,,0
4,"{'id': 'NY.GDP.MKTP.CD', 'value': 'GDP (curren...","{'id': 'US', 'value': 'United States'}",USA,2020,2.135410e+13,,,0


In [2]:
spark_df = spark.createDataFrame(df)
spark_df.show(5)


StatementMeta(, 493837e5-74ea-4355-89f2-530603fe6e7d, 4, Finished, Available, Finished)

+--------------------+-------------------+---------------+----+------------+----+----------+-------+
|           indicator|            country|countryiso3code|date|       value|unit|obs_status|decimal|
+--------------------+-------------------+---------------+----+------------+----+----------+-------+
|{NY.GDP.MKTP.CD, ...|{US, United States}|            USA|2024| 2.918489E13|    |          |      0|
|{NY.GDP.MKTP.CD, ...|{US, United States}|            USA|2023|2.7720709E13|    |          |      0|
|{NY.GDP.MKTP.CD, ...|{US, United States}|            USA|2022|2.6006893E13|    |          |      0|
|{NY.GDP.MKTP.CD, ...|{US, United States}|            USA|2021|2.3681171E13|    |          |      0|
|{NY.GDP.MKTP.CD, ...|{US, United States}|            USA|2020|2.1354105E13|    |          |      0|
+--------------------+-------------------+---------------+----+------------+----+----------+-------+
only showing top 5 rows



In [3]:
from pyspark.sql.functions import col

clean_df = (
    spark_df
    .select(
        col("country.value").alias("country"),
        col("indicator.value").alias("indicator_name"),
        col("date").cast("int").alias("year"),
        col("value").cast("double").alias("indicator_value")
    )
    .dropna(subset=["indicator_value"])
)

clean_df.show(10)


StatementMeta(, 493837e5-74ea-4355-89f2-530603fe6e7d, 5, Finished, Available, Finished)

+-------------+-----------------+----+---------------+
|      country|   indicator_name|year|indicator_value|
+-------------+-----------------+----+---------------+
|United States|GDP (current US$)|2024|    2.918489E13|
|United States|GDP (current US$)|2023|   2.7720709E13|
|United States|GDP (current US$)|2022|   2.6006893E13|
|United States|GDP (current US$)|2021|   2.3681171E13|
|United States|GDP (current US$)|2020|   2.1354105E13|
|United States|GDP (current US$)|2019|   2.1539982E13|
|United States|GDP (current US$)|2018|   2.0656516E13|
|United States|GDP (current US$)|2017|   1.9612102E13|
|United States|GDP (current US$)|2016|   1.8804913E13|
|United States|GDP (current US$)|2015|   1.8295019E13|
+-------------+-----------------+----+---------------+
only showing top 10 rows



In [4]:
clean_df.write.format("delta").mode("overwrite").saveAsTable("worldbank_data")


StatementMeta(, 493837e5-74ea-4355-89f2-530603fe6e7d, 6, Finished, Available, Finished)